In [1]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
distance =  pd.read_csv("hw2.2.1.csv")
demand = pd.read_csv("hw2.2.2.csv")

In [3]:
distance.head()

,0,1,2,3,4,5,6,7,8,9
0,0,20,41,17,18,38,64,37,54,71
1,20,0,21,37,38,18,43,53,36,53
2,41,21,0,58,59,39,22,78,57,46
3,17,37,58,0,35,55,78,20,37,54
4,18,38,59,35,0,20,48,55,38,52


In [4]:
demand.head()

,Points,Demand
0,A,18
1,B,27
2,C,26
3,D,10
4,E,11


In [6]:
#f = demand['cost'].tolist()
h = demand['Demand'].tolist()
n = len(demand)

In [9]:
I = {i for i in range(0,n)}
J = {i for i in range(0,n)}
A = [(i,j) for i in I for j in J]

#Converting distance matrix to dictionary like data type
d = {(i,j): distance.iat[i,j] for i in range(0,n) for j in range(0,n)}


In [10]:
d = dict((k,v) for k,v in d.items() if v<=18)
d

{(0, 0): 0,
 (0, 3): 17,
 (0, 4): 18,
 (1, 1): 0,
 (1, 5): 18,
 (2, 2): 0,
 (3, 0): 17,
 (3, 3): 0,
 (4, 0): 18,
 (4, 4): 0,
 (5, 1): 18,
 (5, 5): 0,
 (5, 8): 18,
 (6, 6): 0,
 (7, 7): 0,
 (7, 8): 17,
 (8, 5): 18,
 (8, 7): 17,
 (8, 8): 0,
 (8, 9): 17,
 (9, 8): 17,
 (9, 9): 0}

In [8]:
#Maximum covering limit problem
mdl = Model('MCLP')

Academic license - for non-commercial use only - expires 2022-08-28
Using license file C:\Users\HP\gurobi.lic


In [16]:
#For p=1
p = 1

In [12]:
facilities, coverage = gp.multidict({0:[{0,3,4}], 1:[{1,5}], 2: [{2}], 3: [{0,3}], 4: [{0,4}], 5: [{1,5,8}], 6: [{6}], 7: [{7,8}], 8: [{7,5,9,8}], 9: [{8,9}]})
coverage

{0: {0, 3, 4},
 1: {1, 5},
 2: {2},
 3: {0, 3},
 4: {0, 4},
 5: {1, 5, 8},
 6: {6},
 7: {7, 8},
 8: {5, 7, 8, 9},
 9: {8, 9}}

In [13]:
x = mdl.addVars(J, vtype = GRB.BINARY)
z = mdl.addVars(I, vtype = GRB.BINARY)

In [15]:
mdl.setObjective(quicksum(h[i]*z[i] for i in I), GRB.MAXIMIZE)

In [17]:
mdl.addConstrs(quicksum(x[j] for j in J if i in coverage[j]) >= z[i] for i in I)
mdl.addConstr(quicksum(x[j] for j in J) == p)

<gurobi.Constr *Awaiting Model Update*>

In [18]:
mdl.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 11 rows, 20 columns and 42 nonzeros
Model fingerprint: 0x0cbc8af6
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 65.0000000
Presolve removed 11 rows and 20 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 116 65 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.160000000000e+02, best bound 1.160000000000e+02, gap 0.0000%


In [19]:
sol_x={j: x[j] for j in J}
sol_x

{0: <gurobi.Var C0 (value 0.0)>,
 1: <gurobi.Var C1 (value 0.0)>,
 2: <gurobi.Var C2 (value 0.0)>,
 3: <gurobi.Var C3 (value 0.0)>,
 4: <gurobi.Var C4 (value 0.0)>,
 5: <gurobi.Var C5 (value 0.0)>,
 6: <gurobi.Var C6 (value 0.0)>,
 7: <gurobi.Var C7 (value 0.0)>,
 8: <gurobi.Var C8 (value 1.0)>,
 9: <gurobi.Var C9 (value 0.0)>}

In [20]:
sol_z={i: z[i] for i in I}
sol_z

{0: <gurobi.Var C10 (value 0.0)>,
 1: <gurobi.Var C11 (value 0.0)>,
 2: <gurobi.Var C12 (value 0.0)>,
 3: <gurobi.Var C13 (value 0.0)>,
 4: <gurobi.Var C14 (value 0.0)>,
 5: <gurobi.Var C15 (value 1.0)>,
 6: <gurobi.Var C16 (value 0.0)>,
 7: <gurobi.Var C17 (value 1.0)>,
 8: <gurobi.Var C18 (value 1.0)>,
 9: <gurobi.Var C19 (value 1.0)>}

In [23]:
#For p=1
p = 2

In [24]:
facilities, coverage = gp.multidict({0:[{0,3,4}], 1:[{1,5}], 2: [{2}], 3: [{0,3}], 4: [{0,4}], 5: [{1,5,8}], 6: [{6}], 7: [{7,8}], 8: [{7,5,9,8}], 9: [{8,9}]})

In [25]:
x = mdl.addVars(J, vtype = GRB.BINARY)
z = mdl.addVars(I, vtype = GRB.BINARY)

In [26]:
mdl.setObjective(quicksum(h[i]*z[i] for i in I), GRB.MAXIMIZE)

In [27]:
mdl.addConstrs(quicksum(x[j] for j in J if i in coverage[j]) >= z[i] for i in I)
mdl.addConstr(quicksum(x[j] for j in J) == p)

<gurobi.Constr *Awaiting Model Update*>

In [28]:
mdl.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 22 rows, 40 columns and 84 nonzeros
Model fingerprint: 0x8d8c4e91
Variable types: 0 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]

MIP start from previous solve produced solution with objective 155 (0.08s)
Loaded MIP start from previous solve with objective 155

Presolve removed 13 rows and 22 columns
Presolve time: 0.00s
Presolved: 9 rows, 18 columns, 38 nonzeros
Variable types: 0 continuous, 18 integer (18 binary)

Root relaxation: cutoff, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       155.00000  155.00000  0.00%     -    0

In [29]:
sol_x={j: x[j] for j in J}
sol_x

{0: <gurobi.Var C20 (value 1.0)>,
 1: <gurobi.Var C21 (value 0.0)>,
 2: <gurobi.Var C22 (value -0.0)>,
 3: <gurobi.Var C23 (value 0.0)>,
 4: <gurobi.Var C24 (value 0.0)>,
 5: <gurobi.Var C25 (value -0.0)>,
 6: <gurobi.Var C26 (value -0.0)>,
 7: <gurobi.Var C27 (value 0.0)>,
 8: <gurobi.Var C28 (value 1.0)>,
 9: <gurobi.Var C29 (value -0.0)>}

## First facility to open is I
## Second facility to open is A

### Result matches the one obtained from ADD Algorithm